# Initial set up

In [ ]:
from bs4 import BeautifulSoup
import unicodedata
import urllib.request,sys,time
import requests
import re
import random
import string
import os
import pandas as pd
import numpy as np

In [ ]:
!pip -q install langdetect
from langdetect import detect

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# DATA FOLDER
data_dir = "/content/gdrive/MyDrive/MLProjects/NLP/central_bank_communications/data"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Riksbank

## Newest

In [ ]:
RIKSBANK_URL = 'https://www.riksbank.se'

In [ ]:
url = os.path.join(RIKSBANK_URL, 'en-gb/press-and-published/speeches-and-presentations/')
cover_page = requests.get(url)
cover_page = cover_page.content

soup = BeautifulSoup(cover_page, 'html5lib')
coverpage_news = soup.find_all(class_="listing-item--speach")
coverpage_news[1]

<a class="listing-item--speach" href="/en-gb/press-and-published/speeches-and-presentations/2023/erik-thedeen-committee-on-finance-hearing-on-the-riksbanks-operations-and-monetary-policy/">
	<h4 class="link-style">Erik Thedèen: Committee on Finance hearing on the Riksbank's operations and monetary policy</h4>
	<p>
		<span class="label">Date: </span><span>14/03/2023</span>
	</p>
	<p>
		<span class="label">Speaker:</span> <span>Erik Thedéen Governor</span>
	</p>

		<p>
			<span class="label">Place:</span> <span>The Committee on Finance</span>
		</p>
</a>

In [ ]:
number_of_articles = 500

urls = []
articles = []
titles = []
dates = []
speakers = []
speaker_titles = []
ids = []

counter = 0
for i in np.arange(0, number_of_articles):
    try:
        # Extract info from soup object
        ## url and articles
        url = os.path.join(RIKSBANK_URL, coverpage_news[i]['href'][1:])
        r = requests.get(url).content
        article_soup = BeautifulSoup(r, 'html5lib')
        main_body = article_soup.find_all(class_="page-base__main__body")  
        article = [unicodedata.normalize("NFKD", p.text) for p in main_body[0].find_all('p')]
        article = [re.sub('\t', '', p) for p in article]
        if (not " ".join(article).strip()) or (detect(" ".join(article)) != 'en'):
            continue
        article = "\n".join(article)

        ## Other info
        title = coverpage_news[i].find('h4').text
        other_info = coverpage_news[i].find_all('span')
        date = other_info[1].text
        speaker = other_info[3].text
        speaker_title = ' '.join(speaker.split()[2:])
        id = 'r' + date.replace("/", "") + speaker[:2].lower() + "".join(random.choices(string.ascii_lowercase, k=2))


        # Save the results
        urls.append(url)
        articles.append(article)
        titles.append(title)
        dates.append(date)
        speakers.append(speaker)
        speaker_titles.append(speaker_title)
        ids.append(id)

        # Counter
        counter += 1

    except IndexError:
        break
print(f'There are {counter} articles')

new_df = pd.DataFrame({'id': ids, 'date':dates, 'title': titles, 'speaker':speakers, 'speaker_title': speaker_titles, 'url': urls, 'article':articles})
new_df['bank'] = "Riksbank"
new_df['policy_rate_control'] = "Riksbank"
new_df['date'] = pd.to_datetime(new_df['date'], format='%d/%m/%Y')
new_df = new_df[['bank', 'policy_rate_control', 'id', 'date', 'speaker', 'speaker_title', 'article']]
new_df

There are 105 articles


,bank,policy_rate_control,id,date,speaker,speaker_title,article
0,Riksbank,Riksbank,r23032023erlg,2023-03-23,Erik Thedéen Governor,Governor,Mr Thedéen began his speech by commenting on ...
1,Riksbank,Riksbank,r14032023erwt,2023-03-14,Erik Thedéen Governor,Governor,Governor Erik Thedéen began the hearing by su...
2,Riksbank,Riksbank,r14032023aiek,2023-03-14,Aino Bunge Deputy Governor,Deputy Governor,Governor Erik Thedéen began the hearing by su...
3,Riksbank,Riksbank,r09032023aibz,2023-03-09,Aino Bunge Deputy Governor,Deputy Governor,“It is a tough situation for many people when ...
4,Riksbank,Riksbank,r31012023erds,2023-01-31,Erik Thedéen Governor,Governor,Erik Thedéen emphasised that significant vuln...
...,...,...,...,...,...,...,...
100,Riksbank,Riksbank,r14032017hemh,2017-03-14,Henry Ohlsson Deputy Governor,Deputy Governor,Mr Ingves will make an introductory presentati...
101,Riksbank,Riksbank,r14032017stti,2017-03-14,Stefan Ingves Governor,Governor,Mr Ingves will make an introductory presentati...
102,Riksbank,Riksbank,r02022017stlr,2017-02-02,Stefan Ingves Governor,Governor,In his role as chairman of the Basel Committee...
103,Riksbank,Riksbank,r31012017kext,2017-01-31,Kerstin af Jochnick First Deputy Governor,Jochnick First Deputy Governor,The Basel Committee has been working on reduci...


## Archive

In [ ]:
RIKSBANK_URL = 'https://archive.riksbank.se/en/Web-archive/Published/Speeches'

In [ ]:
url = os.path.join(RIKSBANK_URL, 'index.html@all=1.html')
cover_page = requests.get(url)
cover_page = cover_page.content

soup = BeautifulSoup(cover_page, 'html5lib')
coverpage_news = soup.find_all('td')
coverpage_news[2]

<td>

    <b><a href="2016/Ingves-Reflections-of-a-Basel-Committee-Chairman/index.html">Ingves: Reflections of a Basel Committee Chairman</a></b>

    <dl class="container">
        <dt>Date</dt>
        <dd>30/11/2016</dd>

        <dt>Speaker</dt>
        <dd>Governor Stefan Ingves</dd>

        <dt>Place</dt>
        <dd>ICBS, Santiago </dd>
    </dl>
    
</td>

In [ ]:
number_of_articles = 1000

urls = []
articles = []
titles = []
dates = []
speakers = []
speaker_titles = []
ids = []

counter = 0
for i in np.arange(0, number_of_articles):
    try:
        # Extract info from soup object
        ## url and articles
        url = os.path.join(RIKSBANK_URL, coverpage_news[i].find('a')['href']) # done
        r = requests.get(url).content
        article_soup = BeautifulSoup(r, 'html5lib')
        main_body = article_soup.find(class_="grid-main")       
        article = [unicodedata.normalize("NFKD", p.text) for p in main_body.find_all('p')]
        article = [re.sub('\t', '', p) for p in article]
        if (not " ".join(article).strip()) or (detect(" ".join(article)) != 'en'):
            continue
        article = "\n".join(article)
        
        ## Other info
        title = coverpage_news[i].find('b').text 
        other_info = coverpage_news[i].find_all('dd') 
        date = other_info[0].text 
        speaker = other_info[1].text 
        speaker_title = ' '.join(speaker.split()[:-2])
        id = 'r' + date.replace("/", "") + speaker[-2:].lower() + "".join(random.choices(string.ascii_lowercase, k=2))


        # Save the results
        urls.append(url)
        articles.append(article)
        titles.append(title)
        dates.append(date)
        speakers.append(speaker)
        speaker_titles.append(speaker_title)
        ids.append(id)

        # Counter
        counter += 1

    except AttributeError:
        continue
    
    except IndexError:
        break

print(f'There are {counter} articles')

archive_df = pd.DataFrame({'id': ids, 'date':dates, 'title': titles, 'speaker':speakers, 'speaker_title': speaker_titles, 'url': urls, 'article':articles})
archive_df['bank'] = "Riksbank"
archive_df['policy_rate_control'] = "Riksbank"
archive_df['date'] = pd.to_datetime(archive_df['date'], format='%d/%m/%Y')
archive_df = archive_df[['bank', 'policy_rate_control', 'id', 'date', 'speaker', 'speaker_title', 'article']]
archive_df

There are 580 articles


,bank,policy_rate_control,id,date,speaker,speaker_title,article
0,Riksbank,Riksbank,r07122016onln,2016-12-07,Deputy Governor Per Jansson,Deputy Governor,"""No"" says Deputy Governor Per Jansson in a spe..."
1,Riksbank,Riksbank,r02122016esbs,2016-12-02,Governor Stefan Ingves,Governor,In his role as chairman of the Basel Committee...
2,Riksbank,Riksbank,r30112016esnz,2016-11-30,Governor Stefan Ingves,Governor,In his role as chairman of the Basel Committee...
3,Riksbank,Riksbank,r16112016eype,2016-11-16,Deputy Governor Cecilia Skingsley,Deputy Governor,Today almost everyone has access to the intern...
4,Riksbank,Riksbank,r15112016ckit,2016-11-15,First Deputy Governor Kerstin af Jochnick,First Deputy Governor Kerstin,"""It is a challenge in Sweden to maintain both ..."
...,...,...,...,...,...,...,...
575,Riksbank,Riksbank,r19031996enkl,1996-03-19,Deputy Governor Lars Heikensten,Deputy Governor,Thank you for inviting me to talk about the Ri...
576,Riksbank,Riksbank,r21021996ömgx,1996-02-21,Governor Urban Bäckström,Governor,A year ago today I had the honour of speaking ...
577,Riksbank,Riksbank,r30011996ensp,1996-01-30,Deputy Governor Lars Heikensten,Deputy Governor,I should like to begin by expressing my thanks...
578,Riksbank,Riksbank,r25011996ömbr,1996-01-25,Governor Urban Bäckström,Governor,Thank you for the invitation to discuss the de...


# Finlands Bank

In [ ]:
FINBANK_URL = 'https://www.suomenpankki.fi'

In [ ]:
url = os.path.join(FINBANK_URL, 'en/media-and-publications/speeches-and-interviews/?y=')
cover_page = requests.get(url)
cover_page = cover_page.content

soup = BeautifulSoup(cover_page, 'html5lib')
coverpage_news = soup.find('article').find_all('li')
coverpage_news[13]

<li>
                                            <h3><a href="/en/media-and-publications/speeches-and-interviews/2022/Resilience-of-borrowers-and-the-financial-system-must-be-strengthened/">Resilience of borrowers and the financial system must be strengthened</a></h3>
                                            <time class="meta">4 May 2022 </time>
                                        </li>

In [ ]:
number_of_articles = 1000

urls = []
articles = []
titles = []
dates = []
speakers = []
speaker_titles = []
ids = []

counter = 0
for i in np.arange(0, number_of_articles):
    try:
        # Extract info from soup object
        ## url and articles
        url = os.path.join(FINBANK_URL, coverpage_news[i].find('h3').find('a')['href'][1:])
        r = requests.get(url).content
        article_soup = BeautifulSoup(r, 'html5lib')
        main_body = article_soup.find('article')      
        article = [unicodedata.normalize("NFKD", p.text) for p in main_body.find_all('p')[1:-1]]
        article = [re.sub('\t', '', p) for p in article]
        if (not " ".join(article).strip()) or (detect(" ".join(article)) != 'en'):
            continue
        article = "\n".join(article)

        ## Other info
        other_infor = coverpage_news[i].find('h3').find('a').text.strip()
        other_infor = other_infor.replace(":",",").split(",")
        title = other_infor[1]
        date = coverpage_news[i].find('time').text.strip()
        speaker = other_infor[0]
        speaker_title = ' '.join(speaker.split()[:-2])
        id = 'fi' + date.replace(" ", "").lower() + speaker[-2:].lower() + "".join(random.choices(string.ascii_lowercase, k=2))


        # Save the results
        urls.append(url)
        articles.append(article)
        titles.append(title)
        dates.append(date)
        speakers.append(speaker)
        speaker_titles.append(speaker_title)
        ids.append(id)

        # Counter
        counter += 1

    except (AttributeError, IndexError):
        continue

print(f'There are {counter} articles')

fi_df = pd.DataFrame({'id': ids, 'date':dates, 'title': titles, 'speaker':speakers, 'speaker_title': speaker_titles, 'url': urls, 'article':articles})
fi_df['bank'] = "Finlands Bank"
fi_df['policy_rate_control'] = "ECB"
fi_df['date'] = pd.to_datetime(fi_df['date'], format="%d %b %Y")
fi_df = fi_df[['bank', 'policy_rate_control', 'id', 'date', 'speaker', 'speaker_title', 'article']]
fi_df

There are 188 articles


,bank,policy_rate_control,id,date,speaker,speaker_title,article
0,Finlands Bank,ECB,fi21feb2023hnig,2023-02-21,Governor Olli Rehn,Governor,"Mr Rehn, has the danger of a recession in the ..."
1,Finlands Bank,ECB,fi11jan2023hnub,2023-01-11,Governor Olli Rehn,Governor,Preparing the economy and financial system for...
2,Finlands Bank,ECB,fi20dec2022hnmt,2022-12-20,Governor Olli Rehn,Governor,It has been said of Sir Winston Churchill that...
3,Finlands Bank,ECB,fi28nov2022hnfi,2022-11-28,Governor Olli Rehn,Governor,European economic policy in the times of crise...
4,Finlands Bank,ECB,fi24nov2022hnvv,2022-11-24,Governor Olli Rehn,Governor,"Public hearing, 22 November 2022 \nThe Europ..."
...,...,...,...,...,...,...,...
183,Finlands Bank,ECB,fi4dec2006encc,2006-12-04,Governor Erkki Liikanen,Governor,\n\nIII High-Level Seminar of the Eurosystem a...
184,Finlands Bank,ECB,fi2nov2006loya,2006-11-02,Member of the Board Sinikka Salo,Member of the Board,This annual event takes place now for the 7th ...
185,Finlands Bank,ECB,fi5sep2006enpb,2006-09-05,Governor Erkki Liikanen,Governor,Monetary policy has no direct connection to la...
186,Finlands Bank,ECB,fi25oct2004enre,2004-10-25,Governor Erkki Liikanen,Governor,Professor Roland's paper raises some essential...


# Save result

In [ ]:
df = pd.concat([new_df, archive_df, fi_df])
df = df.dropna()
df.to_csv(os.path.join(data_dir,'all_corpus.csv'), index=False)
df.head()

,bank,policy_rate_control,id,date,speaker,speaker_title,article
0,Riksbank,Riksbank,r23032023erlg,2023-03-23,Erik Thedéen Governor,Governor,Mr Thedéen began his speech by commenting on ...
1,Riksbank,Riksbank,r14032023erwt,2023-03-14,Erik Thedéen Governor,Governor,Governor Erik Thedéen began the hearing by su...
2,Riksbank,Riksbank,r14032023aiek,2023-03-14,Aino Bunge Deputy Governor,Deputy Governor,Governor Erik Thedéen began the hearing by su...
3,Riksbank,Riksbank,r09032023aibz,2023-03-09,Aino Bunge Deputy Governor,Deputy Governor,“It is a tough situation for many people when ...
4,Riksbank,Riksbank,r31012023erds,2023-01-31,Erik Thedéen Governor,Governor,Erik Thedéen emphasised that significant vuln...
